In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv('cervix.csv',sep= ',')
df.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,18,4,15,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15,1,14,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34,1,?,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,5,16,4,1,37,37,1,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46,3,21,4,0,0,0,1,15,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Columns: 256 entries, Age to Unnamed: 255
dtypes: float64(220), int64(10), object(26)
memory usage: 1.7+ MB


## Data Cleaning

In [5]:
target = 'Dx'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
#df

### Remove bad columns

In [7]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [8]:
df = remove_bad_columns(df,100)
df.info()

number of bad columns: 240
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 16 columns):
Age                          858 non-null int64
Number of sexual partners    832 non-null object
First sexual intercourse     851 non-null object
Num of pregnancies           802 non-null object
Smokes                       845 non-null object
Smokes (years)               845 non-null object
Smokes (packs/year)          845 non-null object
STDs: Number of diagnosis    858 non-null int64
Dx:Cancer                    858 non-null int64
Dx:CIN                       858 non-null int64
Dx:HPV                       858 non-null int64
Dx                           858 non-null int64
Hinselmann                   858 non-null int64
Schiller                     858 non-null int64
Citology                     858 non-null int64
Biopsy                       858 non-null int64
dtypes: int64(10), object(6)
memory usage: 107.3+ KB


### Remove bad rows

In [9]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [10]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [11]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [12]:
bad_rows = find_bad_rows(df,10)
df.info()

number of bad rows: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 16 columns):
Age                          858 non-null int64
Number of sexual partners    832 non-null object
First sexual intercourse     851 non-null object
Num of pregnancies           802 non-null object
Smokes                       845 non-null object
Smokes (years)               845 non-null object
Smokes (packs/year)          845 non-null object
STDs: Number of diagnosis    858 non-null int64
Dx:Cancer                    858 non-null int64
Dx:CIN                       858 non-null int64
Dx:HPV                       858 non-null int64
Dx                           858 non-null int64
Hinselmann                   858 non-null int64
Schiller                     858 non-null int64
Citology                     858 non-null int64
Biopsy                       858 non-null int64
dtypes: int64(10), object(6)
memory usage: 107.3+ KB


In [13]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 0


### Separate target and attributes

In [14]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [15]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[44 12 21 11  2 30 62  4  2  2  2  2  2  2  2]


In [16]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [17]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [4, 8, 9, 10, 11, 12, 13, 14]
i_category: [7]
variable type: [3. 3. 3. 3. 1. 3. 3. 2. 1. 1. 1. 1. 1. 1. 1.]


### Impute missing values of attributes

In [18]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [19]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 858 entries, 0 to 857
Data columns (total 15 columns):
Age                          858 non-null int64
Number of sexual partners    858 non-null object
First sexual intercourse     858 non-null object
Num of pregnancies           858 non-null object
Smokes                       858 non-null object
Smokes (years)               858 non-null object
Smokes (packs/year)          858 non-null object
STDs: Number of diagnosis    858 non-null int64
Dx:Cancer                    858 non-null int64
Dx:CIN                       858 non-null int64
Dx:HPV                       858 non-null int64
Hinselmann                   858 non-null int64
Schiller                     858 non-null int64
Citology                     858 non-null int64
Biopsy                       858 non-null int64
dtypes: int64(9), object(6)
memory usage: 107.2+ KB


## Data Processing

### Attributes

In [20]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [21]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(858, 18)
[[18.  4. 15. ... -1. -1. -1.]
 [15.  1. 14. ... -1. -1. -1.]
 [34.  1. 17. ... -1. -1. -1.]
 ...
 [25.  2. 17. ... -1.  1. -1.]
 [33.  2. 24. ... -1. -1. -1.]
 [29.  2. 20. ... -1. -1. -1.]]


### Target

In [22]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0, 1]), array([834,  24]))


In [23]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('cervix_processed.dat',xy_new,fmt='%f')